# **Centralized Model - UnivEnc**

In [ ]:
!pip install tensorflow_text

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import tensorflow as tf

In [ ]:
get_ipython().system('pip install dgl')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy as sp
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score, average_precision_score, precision_recall_curve
import copy

import dgl
from dgl.data import DGLDataset
import torch

import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
import random
import os
from operator import add

from sklearn.metrics.pairwise import cosine_similarity
import heapq

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# This path contains the CSV file
PATH = '/content/drive/MyDrive/Tweets/IndicDataset_full.csv'

df = pd.read_csv(PATH)

df = df.sample(frac=1).reset_index(drop=True) 
df.head(2)

In [ ]:
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base-br/1")

In [ ]:
labels = df.Label

In [ ]:
sentim = []

for i in range(len(df)):

    if df.Sentiment[i] == 'Negative':
        sentim.append(0)
    if df.Sentiment[i] == 'Neutral':
        sentim.append(1)
    if df.Sentiment[i] == 'Postive':
        sentim.append(2)

In [ ]:
tweets = []
i = 0

for tweet in df['Hinglish']:
    i += 1
    tweets.append(encoder(preprocessor(tf.constant([tweet])))["default"])
    if i % 100 == 0:
        print(i, 'done')

In [ ]:
tweets = np.stack(tweets).reshape((len(df), 768))
df1 = pd.DataFrame(tweets)
df1.head(3)

In [ ]:
# Cosine similarity between the tweet vectors

from sklearn.metrics.pairwise import cosine_similarity
cos_res_temp = cosine_similarity(df1, df1)
print('cos_res_temp:\n', cos_res_temp)
print(cos_res_temp.shape)

In [ ]:
# To ensure that each node has attleast 4 connections
import heapq

for i in range(len(cos_res_temp)):
    ind = heapq.nlargest(4, range(len(cos_res_temp[i])), key=lambda x: cos_res_temp[i][x])
    for j in ind:
        cos_res_temp[i][j] = 1


In [ ]:
# Diagonal of adjacency matrix = Degree of node
cos_res = (cos_res_temp - 0.25).round() - np.identity(len(cos_res_temp))
degree = np.sum(cos_res, axis = 1)

print('Average node degree:', np.round(sum(degree)/len(degree), 3))

In [ ]:
cnt=0
start_indx  = []
end_idx = []
weights = []

for i in range(len(tweets)):
    for j in range(len(tweets)):
        if cos_res[i][j]==1 :
            cnt+=1
            start_indx.append(i)
            end_idx.append(j)
            # weights.append(cos_res_temp[i][j])

In [ ]:
edge_dataFrame = pd.DataFrame({'target':start_indx, 'source':end_idx})
edges_data = pd.DataFrame(weights)
edge_dataFrame.head()

In [ ]:
edge_dataFrame_dic = dict(edge_dataFrame.groupby('target')['source'].apply(list))

In [ ]:
class TweetDataSet(DGLDataset):
    def __init__(self):
        super().__init__(name='tweet_data')

    def process(self):
        #nodes_data = pd.read_csv('./members.csv')
        #edges_data = pd.read_csv('./interactions.csv')
        node_features = torch.from_numpy(df1.values).float()
        node_labels = torch.from_numpy(np.stack(labels))
        #edge_features = torch.from_numpy(edges_data.to_numpy())
        edges_src = torch.from_numpy(edge_dataFrame['source'].to_numpy())
        edges_dst = torch.from_numpy(edge_dataFrame['target'].to_numpy())

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=df1.shape[0])
        self.graph.ndata['feat'] = node_features
        self.graph.ndata['label'] = node_labels
        #self.graph.edata['weight'] = edge_features

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = df1.shape[0]


    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

dataset = TweetDataSet()
graph = dataset[0]

print(graph)

In [ ]:
class GNN(nn.Module):

    def __init__(self, in_feats, out_feats):
        super().__init__()
        
        h1 = 512 * 2
        h2 = 256 * 2
        h3 = 128 * 2
        h4 = 64 * 2
        h5 = 32 * 2

        self.gat1 = dglnn.conv.GATConv(in_feats = in_feats, out_feats = h1, num_heads = 8, attn_drop = 0.6, residual  = True, feat_drop = 0.5)
        self.gat2 = dglnn.conv.GATConv(in_feats = h1, out_feats = h2, num_heads = 12, attn_drop = 0.6, residual  = True, feat_drop = 0.5)
        self.gat3 = dglnn.conv.GATConv(in_feats = h2, out_feats = h3, num_heads = 8, attn_drop = 0.4, residual  = True, feat_drop = 0.5)
        self.gat4 = dglnn.conv.GATConv(in_feats = h3, out_feats = h4, num_heads = 4, attn_drop = 0.4, residual  = True, feat_drop = 0.5)
        self.gat5 = dglnn.conv.GATConv(in_feats = h4, out_feats = h5, num_heads = 8, attn_drop = 0.2, residual  = True, feat_drop = 0.5)
         
        self.dense1 = torch.nn.Linear(in_features = h1 + h2 + h3 + h4 + h5, out_features = h5)
        self.dense2 = torch.nn.Linear(in_features = h5, out_features = 2)
        
        self.dense3 = torch.nn.Linear(in_features =h1 + h2 + h3 + h4 + h5, out_features = h5)
        self.dense4 = torch.nn.Linear(in_features = h5, out_features = 3)

    def forward(self, graph, inputs):

        h1 = self.gat1(graph, inputs)
        h1 = F.relu(h1.mean(dim=1))

        h2 = self.gat2(graph, h1)
        h2 = F.relu(h2.mean(dim=1))
        
        h3 = self.gat3(graph, h2)
        h3 = F.relu(h3.mean(dim=1))

        h4 = self.gat4(graph, h3)
        h4 = F.relu(h4.mean(dim=1))

        h5 = self.gat5(graph, h4)
        h5 = F.relu(h5.mean(dim=1))
    
        h6 = self.dense1(torch.hstack((h1, h2, h3, h4, h5)))
        h6 = self.dense2(F.relu(h6))

        h7 = self.dense3(torch.hstack((h1, h2, h3, h4, h5)))
        h7 = self.dense4(F.relu(h7))

        return [torch.sigmoid(h6), h7]

In [ ]:
node_features = graph.ndata['feat']
node_labels = graph.ndata['label'].long()

n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

sentiment_labels = np.stack(sentim)


In [ ]:
def valid_eval(model, graph, features, labels, mask):

  model.eval()
  
  with torch.no_grad():
      [logits, dummy] = model(graph, features)
      logits = logits[mask]
      labels = labels[mask]
      k, indices = torch.max(logits, dim=1)

      preds = []
      
      for i in logits:
        preds.append(np.argmax(i))

      repo = classification_report(labels, preds, labels=[0, 1])

      f1_0 = int(repo.split('.')[3].split(' ')[0])/100
      f1_1 = int(repo.split('.')[6].split(' ')[0])/100
      
      return f1_0, f1_1

In [ ]:
valid_split = 0.1
test_split = 0.2
epochs = 200
learn_rate = 10e-4
die = 10e-6


def mask_generator(l, k):

    kfold_train_mask = []
    kfold_valid_mask = []

    ld = int((1-test_split) * l)
    valid_split_new = l * valid_split / ld

    for i in range(k):

        dummy = []
        
        dummy = [True for i in range(l)]
        start = int(i*(((1-valid_split_new)*ld)/k))
        dummy[start:(start + int(valid_split_new*ld))] = [False for i in range(int(valid_split_new*ld))]

        dummy[ld:] = [False for i in range(l-ld)]
        kfold_train_mask.append(dummy)

        dummy = []
        
        dummy = [False for i in range(l)]
        dummy[start:(start + int(valid_split_new*ld))] = [True for i in range(int(valid_split_new*ld))]
        
        dummy[ld:] = [False for i in range(l-ld)]
        kfold_valid_mask.append(dummy)

    print('Length of training data:', ld-int(valid_split_new*ld),'(', np.round(100*((ld-int(valid_split_new*ld))/l)), ')', '\nLength of validation data:', int(valid_split_new*ld), '(', np.round((100*valid_split_new*ld)/l), ')' )
    return kfold_train_mask, kfold_valid_mask
        

In [ ]:
def single_task_train(g, model):

    t_acc = []
    v_acc = []
    t_loss = []

    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate, weight_decay=die)
    
    best_f1 = 0
    f10 = []
    f11 = []

    features = g.ndata['feat']
    labels = g.ndata['label']


    train_mask, val_mask = mask_generator(len(labels), epochs)

    for e in range(epochs):
        # Forward
        [logits, dummy] = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask[e]], labels[train_mask[e]])


        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask[e]] == labels[train_mask[e]]).float().mean()
        val_acc = (pred[val_mask[e]] == labels[val_mask[e]]).float().mean()
            
        t_acc.append(train_acc)
        v_acc.append(val_acc)
        t_loss.append(loss)


        f1_0, f1_1 = valid_eval(model, graph, node_features, node_labels, val_mask[e])
        
        f10.append(f1_0)
        f11.append(f1_1)

        if best_f1 < (f1_0 + 2*f1_1)/3:
            best_f1 = (f1_0 + 2*f1_1)/3
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tweets/UnivEnc/UE_Central_Hinglish_SingleTask_Best.pt')

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, train acc: {:.3f} val acc: {:.3f}, f1 of class0: {:.3f}, f1 of class1: {:.3f}, best weighted f1: {:.3f}'.format(
                e, loss, train_acc, val_acc, f1_0, f1_1, best_f1))
        
    return t_acc, v_acc, t_loss, f10, f11

single_task_model = GNN(in_feats = n_features, out_feats = n_labels)

t_acc, v_acc, t_loss, f10, f11 = single_task_train(graph, single_task_model)
torch.save(single_task_model.state_dict(), '/content/drive/MyDrive/Tweets/UnivEnc/UE_Central_Hinglish_SingleTask_Final.pt')


In [ ]:
def multi_task_train(g, model, sentiment_labels):

    t_acc1 = []
    v_acc1 = []
    t_loss1 = []

    t_acc2 = []
    v_acc2 = []
    t_loss2 = []

    sentiment_labels = torch.from_numpy(sentiment_labels)
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate, weight_decay=die)

    best_f1 = 0
    f10m = []
    f11m = []

    features = g.ndata['feat']
    labels = g.ndata['label']


    train_mask, val_mask = mask_generator(len(labels), epochs)

    for e in range(epochs):
        # Forward
        [logits1, logits2] = model(g, features)

        # Compute prediction
        pred1 = logits1.argmax(1)
        pred2 = logits2.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss1 = F.cross_entropy(logits1[train_mask[e]], labels[train_mask[e]])

        criterion = torch.nn.CrossEntropyLoss() # This criterion combines LogSoftmax and NLLLoss in one single class.
        loss2 = criterion(logits2[train_mask[e]], sentiment_labels[train_mask[e]])

        loss = (2*loss1 + loss2)/3


        train_acc1 = (pred1[train_mask[e]] == labels[train_mask[e]]).float().mean()
        val_acc1 = (pred1[val_mask[e]] == labels[val_mask[e]]).float().mean()

        #print(pred2, sentiment_labels)
        train_acc2 = (pred2[train_mask[e]] == sentiment_labels[train_mask[e]]).float().mean()
        val_acc2 = (pred2[val_mask[e]] == sentiment_labels[val_mask[e]]).float().mean()

        t_acc1.append(train_acc1)
        t_acc2.append(train_acc2)

        v_acc1.append(val_acc1)
        v_acc2.append(val_acc2)
        
        t_loss1.append(loss1) 
        t_loss2.append(loss2) 

        # Save the best validation accuracy and the corresponding test accuracy.
        f1_0, f1_1 = valid_eval(model, graph, node_features, node_labels, val_mask[e])

        f10m.append(f1_0)
        f11m.append(f1_1)

        if best_f1 < (f1_0 + 2*f1_1)/3:
            best_f1 = (f1_0 + 2*f1_1)/3
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tweets/UnivEnc/UE_Central_Hinglish_MultiTask_Best.pt')

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if e % 5 == 0:
            print('In epoch {},\nnode loss : {:.3f}, node train acc : {:.3f} node val acc : {:.3f}, f1 of class0: {:.3f}, f1 of class1 : {:.3f}, best weighted f1: {:.3f}'.format(
                e, loss1, train_acc1, val_acc1, f1_0, f1_1, best_f1))
            print('sent loss : {:.3f}, sent train acc : {:.3f} sent val acc : {:.3f}'.format(loss2, train_acc2, val_acc2))
            print('----------------------------------------')            
            
    return t_acc1, v_acc1, loss, t_acc2, v_acc2, f10m, f11m 

multi_task_model = GNN(in_feats = n_features, out_feats = n_labels)

t_acc1, v_acc1, t_loss1, t_acc2, v_acc2, f10m, f11m = multi_task_train(graph, multi_task_model, sentiment_labels)
torch.save(multi_task_model.state_dict(), '/content/drive/MyDrive/Tweets/UnivEnc/UE_Central_Hinglish_MultiTask_Final.pt')

In [ ]:
plt.figure(figsize= (13, 7))

plt.plot(t_acc1)
plt.plot(v_acc1)

plt.plot(t_acc)
plt.plot(v_acc)

plt.legend(['Multi-Train', 'Multi-Valid', 'Single-Train', 'Single-Valid'])

In [ ]:
best_single_task_model = GNN(in_feats = n_features, out_feats = n_labels)
best_single_task_model.load_state_dict(torch.load('/content/drive/MyDrive/Tweets/UnivEnc/UE_Central_Hinglish_SingleTask_Best.pt'))

best_multi_task_model = GNN(in_feats = n_features, out_feats = n_labels)
best_multi_task_model.load_state_dict(torch.load('/content/drive/MyDrive/Tweets/UnivEnc/UE_Central_Hinglish_MultiTask_Best.pt'))

In [ ]:
def evaluate(model, graph, features, labels, mask):

  model.eval()
  
  with torch.no_grad():
      [logits, dummy] = model(graph, features)
      logits = logits[test_mask]
      labels = labels[test_mask]
      k, indices = torch.max(logits, dim=1)

      preds = []
      
      for i in logits:
        preds.append(np.argmax(i))

      print(classification_report(labels, preds, labels=[0, 1]))
      return preds,k


l = len(node_labels)
test_mask = [False for i in range(l)]
test_mask[int((1-test_split) * l):] = [True for i in range(l-int((1-test_split) * l))]

print('Best Model \n')

print('---------------------------')

print('Single Task Model:')
preds, k = evaluate(best_single_task_model, graph, node_features, node_labels, test_mask)

print('---------------------------')

print('Multi Task model:')
preds, k = evaluate(best_multi_task_model, graph, node_features, node_labels, test_mask)


print('---------------------------')

print('\n Final Model \n')
print('Single Task Model:')
preds, k = evaluate(single_task_model, graph, node_features, node_labels, test_mask)

print('---------------------------')

print('Multi Task model:')
preds, k = evaluate(multi_task_model, graph, node_features, node_labels, test_mask)

In [ ]:
best_multi_task_model = GNN(in_feats = n_features, out_feats = n_labels)
best_multi_task_model.load_state_dict(torch.load('/content/drive/MyDrive/Tweets/UnivEnc/UE_Central_Hinglish_MultiTask_Best.pt'))

In [ ]:
def evaluate(model, graph, features, labels, mask):

  model.eval()
  
  with torch.no_grad():
      [logits, dummy] = model(graph, features)
      logits = logits[test_mask]
      labels = labels[test_mask]
      k, indices = torch.max(logits, dim=1)

      preds = []
      
      for i in logits:
        preds.append(np.argmax(i))
      print(classification_report(labels, preds, labels=[0, 1]))
      return preds,k


l = len(node_labels)
test_mask = [False for i in range(l)]
test_mask[int((1-test_split) * l):] = [True for i in range(l-int((1-test_split) * l))]


print('Multi Task model:')
preds, k = evaluate(best_multi_task_model, graph, node_features, node_labels, test_mask)